In [1]:
import numpy as np
import napari
import mrcfile
import tifffile
import tempfile

In [2]:
import urllib.request
def download_file_from_ftpurl(url):
    with urllib.request.urlopen(url) as response:
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(response.read())
            temp_file_path = temp_file.name

    return temp_file_path

In [3]:
#Load data
mrc_fn = download_file_from_ftpurl("https://ftp.ebi.ac.uk/empiar/world_availability/10988/data/VPP/tomograms/TS_0005.rec")

In [4]:
with mrcfile.open(mrc_fn) as f:
    data0=f.data.copy()

In [25]:
print(mrc_fn)
print(data0.max())
print(data0.min())

C:\Users\LUIS-W~1\AppData\Local\Temp\tmph16zd5_z
127
-128


Be careful with negative int values

Get respective cyto-ribossomes and membranes and join together 

In [7]:
mrc_fn_labels = download_file_from_ftpurl("https://ftp.ebi.ac.uk/empiar/world_availability/10988/data/VPP/labels/TS_0005_cyto_ribosomes.mrc")

In [8]:
with mrcfile.open(mrc_fn_labels) as f:
    labels_rib=f.data.copy().astype(np.uint8)

In [9]:
print(mrc_fn_labels)
print(labels_rib.max())
print(labels_rib.min())

C:\Users\LUIS-W~1\AppData\Local\Temp\tmplsd1sf44
1
0


In [10]:
mrc_fn_membr = download_file_from_ftpurl("https://ftp.ebi.ac.uk/empiar/world_availability/10988/data/VPP/labels/TS_0005_membranes.mrc")

In [11]:
with mrcfile.open(mrc_fn_membr) as f:
    labels_memb=f.data.copy().astype(np.uint8)

In [12]:
print(mrc_fn_membr)
print(labels_memb.max())
print(labels_memb.min())

C:\Users\LUIS-W~1\AppData\Local\Temp\tmpek2ny846
1
0


join labels together, but be careful that there is some overlap. In overlap regions use ribos label

ribo=1
memb=2

In [13]:
labels_rib_memb = labels_rib+2*labels_memb


In [14]:
labels_rib_memb.max()

3

In [15]:
labels_rib_memb [labels_rib_memb==3] = 1 #replace overlap

ensure that voxels are not shared, otherwise we would have max2

In [24]:
labels_rib_memb.max()

2

check napari full volume

In [ ]:
NV=napari.Viewer()
NV.add_image(data0)
NV.add_labels(labels_memb)
NV.add_labels(labels_rib)

Choose centre 256 cube

In [16]:
#Adjust centre
c0 = (269,345,390)

In [29]:
data0_crop = data0[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

In [30]:
labels_rib_memb_crop = labels_rib_memb[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

Check in napari

In [27]:
NV=napari.Viewer()
NV.add_image(data0_crop)
NV.add_labels(labels_rib_memb_crop)

<Labels layer 'labels_rib_memb_crop' at 0x161e78a0890>

Save

In [31]:
tifffile.imwrite("TS_0005_crop.tif", data0_crop)
tifffile.imwrite("TS_0005_ribos_membr_crop.tif", labels_rib_memb_crop)

Generate two more files for validation

In [32]:
#Adjust centre
c0 = (269,345,390+256)

In [33]:
data0_crop = data0[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

In [34]:
labels_rib_memb_crop = labels_rib_memb[ c0[0]-128: c0[0]+128,
                   c0[1]-128: c0[1]+128,
                   c0[2]-128: c0[2]+128]

Check in napari

In [26]:
NV=napari.Viewer()
NV.add_image(data0_crop)
NV.add_labels(labels_rib_memb_crop)

<Labels layer 'labels_rib_memb_crop' at 0x161ea50e010>

In [35]:
tifffile.imwrite("TS_0005_crop_val.tif", data0_crop)
tifffile.imwrite("TS_0005_ribos_membr_crop_val.tif", labels_rib_memb_crop)

In [36]:
# delete temp files
import os
os.remove(mrc_fn) 
os.remove(mrc_fn_labels)
os.remove(mrc_fn_membr)